In [5]:
import geopandas as gpd

points = gpd.read_file("kappazunder-points.gpkg", layer="kappazunder_image_punkte")

In [6]:
import numpy as np
from scipy.spatial import cKDTree
from tqdm import tqdm

# Load your points and ensure CRS is EPSG:31256 (meters)
gdf = points

# Extract coordinates as numpy array
coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))

# Shuffle indices for random selection
indices = np.arange(len(coords))
np.random.shuffle(indices)

# Build KDTree
tree = cKDTree(coords)

# Minimum distance threshold
min_dist = 20

# Boolean mask to track kept points
keep = np.ones(len(coords), dtype=bool)

for i in tqdm(indices):
    if keep[i]:
        # Find all points within min_dist of current point
        neighbors = tree.query_ball_point(coords[i], r=min_dist)
        # Mark all neighbors (except current point) as False
        neighbors.remove(i)
        keep[neighbors] = False

# Select sampled points
sampled_gdf = gdf[keep].copy()

# Save result
print(f"Original points: {len(gdf)}, Sampled points: {len(sampled_gdf)}")


100%|██████████| 1371422/1371422 [00:01<00:00, 1354581.71it/s]

Original points: 1371422, Sampled points: 111496


In [9]:
import numpy as np
from tqdm import tqdm

tqdm.pandas()

# Create combined index column for fast lookup
points["index_key"] = (
    points["trajectoryid"].astype(str) + "_" + points["objectid"].astype(str)
)

# Set this as the index for O(1) lookups
points_indexed = points.set_index("index_key")

# Calculate direction of trajectory at each point
# Direction is calculated from previous point (objectid - 1) to next point (objectid + 1)
# Falls back to one-sided directions if one endpoint is missing


def calculate_direction(row, points_indexed):
    """
    Calculate direction in degrees (0-360).
    Priority:
    1. Direction from previous to next point (if both exist)
    2. Direction from previous to current point (if next doesn't exist)
    3. Direction from current to next point (if previous doesn't exist)
    Returns NaN if neither previous nor next point exists.
    """
    objectid = row["objectid"]
    trajectoryid = row["trajectoryid"]
    current_point = row["geometry"]

    # Get previous and next objectids
    prev_objectid = str(int(float(objectid)) - 1)
    next_objectid = str(objectid + 1)

    # Create keys for lookup
    prev_key = trajectoryid + "_" + prev_objectid
    next_key = trajectoryid + "_" + next_objectid

    # Try to get points from index
    try:
        prev_point = points_indexed.loc[prev_key, "geometry"]
    except KeyError:
        prev_point = None

    try:
        next_point = points_indexed.loc[next_key, "geometry"]
    except KeyError:
        next_point = None

    # Primary: both previous and next exist
    if prev_point is not None and next_point is not None:
        dx = next_point.x - prev_point.x
        dy = next_point.y - prev_point.y
    # Fallback 1: only next exists
    elif next_point is not None:
        dx = next_point.x - current_point.x
        dy = next_point.y - current_point.y
    # Fallback 2: only previous exists
    elif prev_point is not None:
        dx = current_point.x - prev_point.x
        dy = current_point.y - prev_point.y
    # Neither exists
    else:
        return np.nan

    # Calculate angle in degrees (0-360)
    angle = np.degrees(np.arctan2(dy, dx))
    # Normalize to 0-360 range
    angle = (angle + 360) % 360

    return angle


# Calculate direction for each point
print("Calculating directions...")
sampled_gdf["direction"] = sampled_gdf.progress_apply(
    lambda row: calculate_direction(row, points_indexed), axis=1
)

print(
    f"Direction calculated. NaN values (points without any prev/next): {sampled_gdf['direction'].isna().sum()}"
)

Calculating directions...


100%|██████████| 111496/111496 [00:03<00:00, 30135.88it/s]

Direction calculated. NaN values (points without any prev/next): 17


In [10]:
points["objectid"] = points["objectid"].astype(int).astype(str)

In [11]:
sampled_gdf = sampled_gdf.set_crs(epsg=31256).to_crs(epsg=4326)

In [12]:
sampled_gdf.to_file("sampled-points.gpkg", driver="GPKG", layer="kappazunder_image_punkte")